### Base CONV for YOLO

In [3]:
import torch
import torch.nn as nn

def get_conv2d(c1, c2, k, p, s, d, g, bias=False):
    conv = nn.Conv2d(c1, c2, k, padding=p, stride=s, dilation=d, groups=g, bias=bias)
    return conv

def get_activation(act_type=None):
    if act_type == 'relu':
        return nn.ReLU(inplace=True)
    elif act_type == 'lrelu':
        return nn.LeakyReLU(inplace=True)
    elif act_type == 'mish':
        return nn.Mish(inplace=True)
    elif act_type == 'silu':
        return nn.SiLU(inplace=True)
    
def get_norm(norm_type, dim):
    if norm_type == 'BN':
        return nn.BatchNorm2d(dim)
    


In [4]:
class Conv(nn.Module):
    def __init__(self, c1, c2, k=1, p=0, s=0, d=1, act_type='relu', norm_type='BN', depthwise=False):
        super(Conv, self).__init__()
        convs = []
        add_bias = False if norm_type else True 
        convs.append(get_conv2d(c1, c2, k=k, p=p, d=d, g=c1, bias=add_bias))

        if norm_type:
            convs.append(get_norm(norm_type, c1)) # batch normalization
        if act_type:
            convs.append(get_activation(act_type)) # activation

        if depthwise:
            # pointwise
            convs.append(get_conv2d(c1, c2, k=k, p=p, s=s, d=d, g=c1, bias=add_bias))
            if norm_type:
                convs.append(get_norm(norm_type, c1)) # batch normalization
            if act_type:
                convs.append(get_activation(act_type)) # activation

        self.convs = nn.Sequential(*convs)

    def forward(self, x):
        return self.convs(x)

    

In [5]:
class Conv_BN_LeakyReLU(nn.Module):
    def __init__(self, input, output, kernel_size, padding=0, stride=1, dilation=1):
        super(Conv_BN_LeakyReLU, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(input, output, kernel_size, padding=padding, stride=stride, dilation=dilation),
            nn.BatchNorm2d(output),
            nn.LeakyReLU(0.1, inplace=True)
        )

    def forward(self, x):
        return self.convs(x)

In [6]:
class Darknet(nn.Module):
    def __init__(self):
        super(Darknet, self).__init__()
        self.conv_1 = nn.Sequential(
            Conv_BN_LeakyReLU(3, 32, kernel_size=3, stride=1),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.conv_2 = nn.Sequential(
            Conv_BN_LeakyReLU(32, 64, kernel_size=3, stride=1),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.conv_3 = nn.Sequential(
            Conv_BN_LeakyReLU(64, 128, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(128, 64, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(64, 128, kernel_size=3, stride=1),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.conv_4 = nn.Sequential(
            Conv_BN_LeakyReLU(128, 256, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(256, 128, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(128, 256, kernel_size=3, stride=1),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.conv_5 = nn.Sequential(
            Conv_BN_LeakyReLU(256, 512, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(512, 256, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(256, 512, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(512, 256, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(256, 512, kernel_size=3, stride=1),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.conv_6 = nn.Sequential(
            Conv_BN_LeakyReLU(512, 1024, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(1024, 512, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(512, 1024, kernel_size=3, stride=1),
            Conv_BN_LeakyReLU(1024, 512, kernel_size=1, stride=1),
            Conv_BN_LeakyReLU(512, 1024, kernel_size=3, stride=1),
        )

    def forward(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.conv_6(x)
        return x


### YOLOv2 Backbone

In [7]:
class YOLOv2Backbone(nn.Module):
    def __init__(self, n_classes=2):
        super(YOLOv2Backbone, self).__init__()
        self.backbone = Darknet()
        self.flatten = nn.Flatten()
        self.classifier = nn.Linear(in_features=1024, out_features=n_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [9]:
model = YOLOv2Backbone(2).to('cpu')

In [10]:
model

YOLOv2Backbone(
  (backbone): Darknet(
    (conv_1): Sequential(
      (0): Conv_BN_LeakyReLU(
        (convs): Sequential(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.1, inplace=True)
        )
      )
      (1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv_2): Sequential(
      (0): Conv_BN_LeakyReLU(
        (convs): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.1, inplace=True)
        )
      )
      (1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv_3): Sequential(
      (0): Conv_BN_LeakyReLU(
        (convs): Sequential(
          (0): Conv2d(64, 128, kernel_s